In [1]:
# Import our dependencies
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

data = Path('Resourses/patient_data.csv')
df = pd.read_csv(data)
df.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,...,glyburide-metformin:Up,A1Cresult:>7,A1Cresult:>8,A1Cresult:None,A1Cresult:Norm,max_glu_serum:>200,max_glu_serum:>300,max_glu_serum:None,max_glu_serum:Norm,readmitted
0,4,66,3,18,0,0,1,9,1,1,...,0,0,0,1,0,0,0,1,0,1
1,2,48,0,15,4,0,0,7,0,0,...,0,0,0,1,0,0,0,1,0,0
2,4,21,3,23,1,0,2,7,0,1,...,0,0,0,1,0,0,0,1,0,0
3,5,38,0,5,0,0,0,2,1,1,...,0,0,1,0,0,0,0,1,0,0
4,1,6,0,6,0,0,0,6,0,1,...,0,0,0,1,0,0,0,1,0,1


In [2]:
# Split target column from dataset
y = df['readmitted']
X = df.drop(columns='readmitted')

In [10]:
# Use sklearn to split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create scaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # match the number of input layers
    input_features = len(X_train.columns)

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Input(shape=[input_features]))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 20)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=20,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [19]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [20]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 02m 04s]
val_accuracy: 0.623835563659668

Best val_accuracy So Far: 0.6331512331962585
Total elapsed time: 01h 34m 02s


In [21]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'num_layers': 9,
 'units_0': 5,
 'units_1': 19,
 'units_2': 11,
 'units_3': 9,
 'units_4': 9,
 'units_5': 7,
 'units_6': 15,
 'units_7': 15,
 'units_8': 3,
 'units_9': 11,
 'units_10': 11,
 'units_11': 5,
 'units_12': 9,
 'units_13': 13,
 'units_14': 3,
 'units_15': 5,
 'units_16': 9,
 'units_17': 3,
 'units_18': 9,
 'units_19': 17,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 17,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0136'}

In [22]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

796/796 - 1s - 892us/step - accuracy: 0.6332 - loss: 0.6442
Loss: 0.6441702246665955, Accuracy: 0.6331512331962585


In [23]:
# Train the model
fit_model = best_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 4s 854us/step - accuracy: 0.6419 - loss: 0.6302
Epoch 2/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 842us/step - accuracy: 0.6413 - loss: 0.6294
Epoch 3/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 862us/step - accuracy: 0.6451 - loss: 0.6275
Epoch 4/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 863us/step - accuracy: 0.6427 - loss: 0.6293
Epoch 5/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 866us/step - accuracy: 0.6438 - loss: 0.6279
Epoch 6/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 858us/step - accuracy: 0.6419 - loss: 0.6304
Epoch 7/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 849us/step - accuracy: 0.6453 - loss: 0.6276
Epoch 8/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 841us/step - accuracy: 0.6440 - loss: 0.6283
Epoch 9/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 860us/step - accuracy: 0.6430 - loss: 0.6277
Epoch 10/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 871us/step - accuracy: 0.6452 - loss: 0.6269
Epoch 11/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 845us/step - accuracy: 0.6438 - loss: 0.62

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Accuracy: {model_accuracy}, Loss: {model_loss}")

796/796 - 1s - 633us/step - accuracy: 0.6284 - loss: 0.6509
Accuracy: 0.628434419631958, Loss: 0.6508653163909912


In [25]:
# Export our model to HDF5 file
best_model.save("NN_1.h5")